In [1]:
from joblib import Parallel, delayed

In [1]:
import ARX_Optimization as ID
import pandas
import numpy as np
import xlsxwriter
import pathlib
import configparser
from config import datadir

In [2]:
config = configparser.ConfigParser()
config.read('config.ini')
datadir = pathlib.Path(config['paths']['datadir']).expanduser()
inputs = pandas.read_csv(datadir/'Step1.csv')
# results.to_csv(datadir/'results.csv') !!NB

In [3]:
# specify the order:
m, n = 2, 2

# initialize bounds:
bnds = ((-1,1),(-1,1),(-1,1),(-1,1))

# # Adding 10 degrees of noise:
# otpts, std_dev = ID.add_noise(ydata1, 5)

# # Changing the shape of the data in order to simply write it into the excel file later.
# rewrite_noise_data = []
# for j in range(0, len(ydata1)):
#     rewrite = [item[j] for item in otpts]
#     rewrite_noise_data.append(rewrite) 

def parallel_runs(N):
    
    # Adding 10 degrees of noise:
    otpts, std_dev = ID.add_noise(ydata1, 5)

    # Changing the shape of the data in order to simply write it into the excel file later.
    rewrite_noise_data = []
    for j in range(0, len(ydata1)):
        rewrite = [item[j] for item in otpts]
        rewrite_noise_data.append(rewrite) 

    # Now to get the runtimes, coefficients and errors of the data:
    coefficients = []
    rts = []
    for i in range(0,N):
        ni = otpts[i,:]
        runtimes, coeffs = ID.DE_rt_coeffs(bnds, ni, udata1, m, n)
        coefficients.append(coeffs)
        rts.append(runtimes)
    return otpts, std_dev, coefficients, rts


In [73]:
# The for loop that performs identification is run in parallel ten times to obtain possible trends. 
block = Parallel(-1)(delayed(parallel_runs)(10) for i in range(0, 10))

In [74]:
block[0]

(array([[20.9495    , 20.9495    , 20.9495    , ..., 50.9234    ,
         50.9234    , 50.9234    ],
        [21.9495    , 21.9495    , 21.9495    , ..., 51.9234    ,
         51.9234    , 51.9234    ],
        [22.15576032, 21.90736325, 22.61708411, ..., 51.10005881,
         51.45410928, 51.64707117],
        ...,
        [24.5777407 , 24.22128841, 22.68105893, ..., 52.96886231,
         48.84528313, 56.72873505],
        [20.19737172, 19.24845828, 30.06100961, ..., 46.68776569,
         48.63668015, 45.43253246],
        [26.77198965, 29.26924669, 13.75582426, ..., 58.5076801 ,
         58.3626542 , 56.8973402 ]]),
 array([0.        , 0.55555556, 1.11111111, 1.66666667, 2.22222222,
        2.77777778, 3.33333333, 3.88888889, 4.44444444, 5.        ]),
 [array([5.06607526e-01, 4.85119287e-01, 2.04772281e-08, 5.05748849e-03]),
  array([ 5.07211645e-01,  4.84518438e-01, -5.77115234e-08,  5.05555450e-03]),
  array([0.48489205, 0.50428331, 0.00349893, 0.00277536]),
  array([ 0.4940961 , 

In [68]:
workbook = xlsxwriter.Workbook('TCL_ARX_order2_DIFEV.xlsx')
worksheet1 = workbook.add_worksheet('Data')
worksheet2 = workbook.add_worksheet('Results')

bold = workbook.add_format({'bold': True})
runcount = 0
row = 1
for run in block:
    coefficients = run[0]
    runtimes = run[1]
    for set_, tms in zip(rewrite_noise_data, runtimes):
        worksheet1.write(row, 0, 'run'+str(runcount), bold)
        worksheet2.write(row, 0, 'run'+str(runcount), bold)
        col = 1
        devcount = 0
        for value, coeff in zip(set_, coefficients):
            worksheet1.write(0, col, 'output_'+str(devcount), bold) 
            worksheet1.write(row, col, value)
            coeffcol = 1
            ct_a = 1
            ct_b = 1
            devcount += 1
            for a in coeff:
                if coeffcol <= m:
                    worksheet2.write(0, coeffcol, 'a'+str(ct_a), bold)
                    ct_a +=1
                else:
                    worksheet2.write(0, coeffcol, 'b'+str(ct_b), bold)
                    ct_b +=1

                worksheet2.write(row, coeffcol, a)
                coeffcol+=1
            col +=1
        worksheet2.write(0, coeffcol, 'Runtime', bold)
        worksheet2.write(row, coeffcol, tms)
        row +=1



#     for a,b in zip(coefficients, rts):
#         worksheet.write(row, 0, 'run'+str(runcount), bold) 
#         col = 1
#         ct1 = 1
#         ct2 = 1
#         for j in a:
#             if col <= m-1:
#                 worksheet.write(0, col, 'a'+str(ct1), bold)
#                 ct1 +=1
#             else:
#                 worksheet.write(0, col, 'b'+str(ct2), bold)
#                 ct2 +=1

#             worksheet.write(row, col, j)
#             col +=1
#         worksheet.write(0, col, 'Runtime', bold)    
#         worksheet.write(row, col, b)
#         row +=1
        
    runcount+=1

workbook.close()


In [60]:
for run in block:
    coefficients = run[0]
    runtimes = run[1]
    for coeff, tms in zip(coefficients, runtimes):
        print(coeff, tms)

[ 5.11063307e-01  4.80687433e-01 -1.01037516e-06  5.04383323e-03] 75.937848
[ 5.06607812e-01  4.85119003e-01 -3.62068331e-09  5.05750654e-03] 79.86871669999998
[0.5071819  0.48141262 0.00283649 0.0036756 ] 53.947686799999985
[ 0.50447048  0.47602051 -0.01929843  0.02954507] 48.7064378
[ 0.48637081  0.48078438 -0.00194346  0.01914321] 40.18455739999996
[ 0.4815331   0.46118521 -0.12582555  0.15226095] 37.098532000000034
[ 0.48350371  0.43529687 -0.08842793  0.12804942] 32.40773680000001
[ 0.44636961  0.45004769 -0.02480264  0.07364641] 29.52999729999999
[ 0.42877194  0.44907929 -0.08206519  0.14636969] 36.041881699999976
[ 0.40716093  0.43095875 -0.03907818  0.11206104] 30.348589199999992
[ 5.06606398e-01  4.85120315e-01 -2.41472622e-07  5.05780280e-03] 79.04720979999999
[ 5.06608031e-01  4.85118778e-01 -4.24865773e-08  5.05755357e-03] 77.97038260000001
[0.50718185 0.48141268 0.00283646 0.00367562] 52.41062679999999
[ 0.50446509  0.47602647 -0.01929962  0.0295459 ] 49.653964
[ 0.4864098

In [58]:
alpha = Parallel()(delayed(stuff)(10) for i in range(0, 2))

In [59]:
alpha

[[2, 4, 6, 8, 10, 12, 14, 16, 18, 20], [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]]

In [50]:
alpha[0]

[2, 4, 6, 8, 10, 12, 14, 16, 18, 20]